In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import imageio
import os
import math
import folium
import seaborn

In [ ]:
pd.options.display.float_format = '{:.6f}'.format
mta = pd.read_csv('./cleaned/cleaned_2020.csv', parse_dates=['DATE'])
mta.set_index(['DATE'], inplace=True)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    daily_mta = mta.loc[mta.index.unique()[0], :]
    display(daily_mta)

In [ ]:
map = folium.Map(location=[40.74, -73.93], zoom_start=10.2)
for index, row in mta.loc[mta.index.unique()[0], :].iterrows():
    folium.CircleMarker([row['LATITUDE'], row['LONGITUDE']], radius=math.log1p(row['ENTRIES_DIFF']),
                        tooltip=row['STATION']+'\n'+str(row['ENTRIES_DIFF']), fill=True).add_to(map)
map

In [ ]:
map.save('map.html')

In [ ]:
daily_mta = mta.loc[mta.index.unique()[0], :]
geo_mta = gpd.GeoDataFrame(daily_mta, geometry=gpd.points_from_xy(daily_mta['LONGITUDE'], daily_mta['LATITUDE']))
geo_mta.crs = 'epsg:4326'
geo_mta = geo_mta.to_crs(epsg=3857)
ax = geo_mta.plot(figsize=(10, 10), color='red', alpha=0.4, markersize=geo_mta['ENTRIES_DIFF']/120)
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)
ax.set_title('MTA Ridership by Number of Entries', fontdict={'fontsize': '20'})
ax.set_axis_off()
plt.savefig('./test.png', dpi=100, bbox_inches='tight')

In [ ]:
# Batch process

pd.options.display.float_format = '{:.6f}'.format

images = []
mta = pd.read_csv('./cleaned/cleaned_2020.csv', parse_dates=['DATE'])
mta.set_index(['DATE'], inplace=True)

for date in mta.index.unique():
    print('\tProcessing '+date.strftime('%m/%d/%Y'))
    daily_mta = mta.loc[date, :]
    geo_mta = gpd.GeoDataFrame(daily_mta, geometry=gpd.points_from_xy(daily_mta['LONGITUDE'], daily_mta['LATITUDE']))
    geo_mta.crs = 'epsg:4326'
    geo_mta = geo_mta.to_crs(epsg=3857)
    ax = geo_mta.plot(figsize=(10, 10), color='red', alpha=0.4, markersize=geo_mta['ENTRIES_DIFF']/120)
    ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite, zoom='auto')
    ax.set_title('MTA Ridership by Number of Entries\n'+date.strftime('%m/%d/%Y'), fontdict={'fontsize': '20'})
    ax.set_axis_off()
    plt.savefig('./maps/'+date.strftime('%y%m%d')+'.jpg', dpi=100, bbox_inches='tight')
    plt.close()
print('Done')